In [46]:
import os
import cv2
import numpy as np
import matplotlib; matplotlib.use('agg')
import matplotlib.pyplot as plt


In [47]:
def split_triptych(path):

    # Read the image
    try:
        # Try using plt.imread first
        img = plt.imread(path)
    except:
        try:
            import urllib.request
            resp = urllib.request.urlopen(path)
            img = np.asarray(bytearray(resp.read()), dtype="uint8")
            img = cv2.imdecode(img, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            raise ValueError(f"Could not load image from {path}")

    # Get the height and width of the image
    height, width = img.shape[:2]

    # Calculate the height of each panel (assuming equal sizes)
    panel_height = height // 3

    # Split the image into three panels (B, G, R from top to bottom)
    # Ensure each panel has exactly the same shape
    blue_channel = img[:panel_height, :]
    green_channel = img[panel_height:2*panel_height, :]
    red_channel = img[2*panel_height:3*panel_height, :]  # Use exact range to ensure same shape

    # Print shapes for debugging
    print(f"Blue channel shape: {blue_channel.shape}")
    print(f"Green channel shape: {green_channel.shape}")
    print(f"Red channel shape: {red_channel.shape}")

    # Check if all channels have the same dimensions
    if blue_channel.shape != green_channel.shape or blue_channel.shape != red_channel.shape:
        # Resize all to the smallest shape to ensure compatibility
        min_height = min(blue_channel.shape[0], green_channel.shape[0], red_channel.shape[0])
        min_width = min(blue_channel.shape[1], green_channel.shape[1], red_channel.shape[1])

        blue_channel = blue_channel[:min_height, :min_width]
        green_channel = green_channel[:min_height, :min_width]
        red_channel = red_channel[:min_height, :min_width]

        print(f"Resized all channels to shape: {blue_channel.shape}")

    # If the channels are grayscale (only 2D), expand them to 3D for stacking
    if len(blue_channel.shape) == 2:
        # Stack the channels in RGB order
        color_image = np.stack((red_channel, green_channel, blue_channel), axis=2)
    else:
        # If channels already have color dimensions, extract first channel only
        red_intensity = red_channel[:,:,0] if len(red_channel.shape) > 2 else red_channel
        green_intensity = green_channel[:,:,0] if len(green_channel.shape) > 2 else green_channel
        blue_intensity = blue_channel[:,:,0] if len(blue_channel.shape) > 2 else blue_channel

        # Stack the intensity channels
        color_image = np.stack((red_intensity, green_intensity, blue_intensity), axis=2)

    # If the image has values in range [0,1], convert to [0,255] for saving
    if color_image.max() <= 1.0:
        color_image = (color_image * 255).astype(np.uint8)

    # Save the color image
    plt.imsave('color_image_task1.jpg', color_image)

    return red_channel, green_channel, blue_channel, color_image


In [48]:
def normalized_cross_correlation(channel1, channel2):
    """
    Compute the normalized cross-correlation between two channels.
    """
    # Flatten the channels to 1D arrays
    channel1_flat = channel1.flatten()
    channel2_flat = channel2.flatten()

    # Normalize the channels by their L2 norm
    norm1 = np.sqrt(np.sum(channel1_flat**2))
    norm2 = np.sqrt(np.sum(channel2_flat**2))

    # Avoid division by zero
    if norm1 == 0 or norm2 == 0:
        return 0

    channel1_norm = channel1_flat / norm1
    channel2_norm = channel2_flat / norm2

    # Compute the dot product (correlation)
    ncc = np.dot(channel1_norm, channel2_norm)

    return ncc

In [ ]:
def normalized_cross_correlation_center(channel1, channel2, border=15):
    """
    Compute the normalized cross-correlation between two channels,
    using only the center region unaffected by offsetting artifacts.
    """
    # Exclude border pixels
    h, w = channel1.shape[:2]
    center1 = channel1[border:h-border, border:w-border]
    center2 = channel2[border:h-border, border:w-border]

    return normalized_cross_correlation(center1, center2)

In [49]:
def simple_dot_product(channel1, channel2):
    """
    Compute the simple dot product between two channels with no normalization.
    """
    # Flatten the channels to 1D arrays
    channel1_flat = channel1.flatten()
    channel2_flat = channel2.flatten()

    # Compute the dot product (correlation)
    dot_product = np.dot(channel1_flat, channel2_flat)

    return dot_product

In [50]:
def best_offset_ncc(fixed_channel, moving_channel, offset_range=15, use_center=True):
    """
    Look for the best offset using normalized cross-correlation.
    """
    best_corr = -1  # Initialize with a negative correlation
    best_y, best_x = 0, 0

    # Try different offsets
    for y_offset in range(-offset_range, offset_range + 1):
        for x_offset in range(-offset_range, offset_range + 1):
            # Shift the moving channel
            shifted = np.roll(moving_channel, (y_offset, x_offset), axis=(0, 1))

            # Compute correlation
            if use_center:
                corr = normalized_cross_correlation_center(fixed_channel, shifted, border=offset_range)
            else:
                corr = normalized_cross_correlation(fixed_channel, shifted)

            # Update if better correlation found
            if corr > best_corr:
                best_corr = corr
                best_y, best_x = y_offset, x_offset

    return best_y, best_x

In [51]:
def best_offset_dot(fixed_channel, moving_channel, offset_range=15, use_center=True):
    """
    best offset using simple dot product.
    """
    best_corr = -float('inf')  # Initialize with negative infinity
    best_y, best_x = 0, 0

    # Try different offsets
    for y_offset in range(-offset_range, offset_range + 1):
        for x_offset in range(-offset_range, offset_range + 1):
            # Shift the moving channel
            shifted = np.roll(moving_channel, (y_offset, x_offset), axis=(0, 1))

            # Compute simple dot product
            if use_center:
                h, w = fixed_channel.shape[:2]
                border = offset_range
                fixed_center = fixed_channel[border:h-border, border:w-border]
                shifted_center = shifted[border:h-border, border:w-border]
                corr = simple_dot_product(fixed_center, shifted_center)
            else:
                corr = simple_dot_product(fixed_channel, shifted)

            # Update if better correlation found
            if corr > best_corr:
                best_corr = corr
                best_y, best_x = y_offset, x_offset

    return best_y, best_x


In [52]:
def align_and_combine(red_channel, green_channel, blue_channel, offset_range=15, use_center=True):
    """
    Align and combine the three channels into a color image using normalized cross-correlation.
    """
    # Find best offsets for green and blue channels relative to red
    g_y_offset, g_x_offset = best_offset_ncc(red_channel, green_channel, offset_range, use_center)
    b_y_offset, b_x_offset = best_offset_ncc(red_channel, blue_channel, offset_range, use_center)

    # Apply the offsets
    aligned_green = np.roll(green_channel, (g_y_offset, g_x_offset), axis=(0, 1))
    aligned_blue = np.roll(blue_channel, (b_y_offset, b_x_offset), axis=(0, 1))

    # Stack the channels
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    # If the image has values in range [0,1], convert to [0,255] for display
    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    # Store the offsets
    offsets = {
        'green': (g_y_offset, g_x_offset),
        'blue': (b_y_offset, b_x_offset)
    }

    return aligned_image, offsets

def align_and_combine_simple_dot(red_channel, green_channel, blue_channel, offset_range=15, use_center=True):
    """
    Align and combine the three channels using simple dot product metric.
    """
    # Find best offsets for green and blue channels relative to red
    g_y_offset, g_x_offset = best_offset_dot(red_channel, green_channel, offset_range, use_center)
    b_y_offset, b_x_offset = best_offset_dot(red_channel, blue_channel, offset_range, use_center)

    # Apply the offsets
    aligned_green = np.roll(green_channel, (g_y_offset, g_x_offset), axis=(0, 1))
    aligned_blue = np.roll(blue_channel, (b_y_offset, b_x_offset), axis=(0, 1))

    # Stack the channels
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    # If the image has values in range [0,1], convert to [0,255] for display
    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    # Store the offsets
    offsets = {
        'green': (g_y_offset, g_x_offset),
        'blue': (b_y_offset, b_x_offset)
    }

    return aligned_image, offsets


In [53]:
def create_pyramid(image, levels=3, scale_factor=4):
    """
    Create an image pyramid by downsampling the image.
    """
    pyramid = [image]
    current_image = image

    for _ in range(levels - 1):
        h, w = current_image.shape[:2]
        new_h, new_w = h // scale_factor, w // scale_factor
        resized = cv2.resize(current_image, (new_w, new_h))
        pyramid.append(resized)

    # Reverse so that lowest resolution is first
    return pyramid[::-1]

def pyramid_align(red_channel, green_channel, blue_channel, levels=3, scale_factor=4, offset_range=10, use_center=True):
    """
    Align RGB channels using an image pyramid and multi-scale approach.
    """

    # Create pyramids
    red_pyramid = create_pyramid(red_channel, levels, scale_factor)
    green_pyramid = create_pyramid(green_channel, levels, scale_factor)
    blue_pyramid = create_pyramid(blue_channel, levels, scale_factor)

    # Initialize offsets at the coarsest level
    g_offset = [0, 0]
    b_offset = [0, 0]

    for level in range(levels):
        scale = scale_factor ** (levels - level - 1)

        # Get current level images
        red = red_pyramid[level]
        green = green_pyramid[level]
        blue = blue_pyramid[level]

        # Apply previous offsets to moving channels, scaled for current level
        green_shifted = np.roll(green, (g_offset[0], g_offset[1]), axis=(0, 1))
        blue_shifted = np.roll(blue, (b_offset[0], b_offset[1]), axis=(0, 1))

        # Refine offsets at current level
        g_dy, g_dx = best_offset_ncc(red, green_shifted, offset_range, use_center)
        b_dy, b_dx = best_offset_ncc(red, blue_shifted, offset_range, use_center)

        # Update global offsets (scale back to full resolution)
        g_offset[0] += g_dy * scale
        g_offset[1] += g_dx * scale
        b_offset[0] += b_dy * scale
        b_offset[1] += b_dx * scale

    # Apply final offsets at full resolution
    aligned_green = np.roll(green_channel, (g_offset[0], g_offset[1]), axis=(0, 1))
    aligned_blue = np.roll(blue_channel, (b_offset[0], b_offset[1]), axis=(0, 1))

    # Combine into final image
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    offsets = {
        'green': tuple(g_offset),
        'blue': tuple(b_offset)
    }

    return aligned_image, offsets




In [54]:
# Main function to process all tasks
def main():
    # Choose an image from the prokudin-gorskii folder
    # You would need to replace this with the actual path or URL
    image_path = 'https://github.com/Mian56/MV_mod4/blob/main/prokudin-gorskii/prokudin-gorskii/00351v.jpg?raw=true'

    print(f"Processing image: {image_path}")

    # Task 1: Split and combine
    red_channel, green_channel, blue_channel, basic_color_image = split_triptych(image_path)

    # Task 2: Align using two different metrics
    aligned_ncc, offsets_ncc = align_and_combine(
        red_channel, green_channel, blue_channel, offset_range=15, use_center=True
    )

    aligned_dot, offsets_dot = align_and_combine_simple_dot(
        red_channel, green_channel, blue_channel, offset_range=15, use_center=True
    )

    # Save the aligned images
    plt.imsave('aligned_image_ncc.jpg', aligned_ncc)
    plt.imsave('aligned_image_dot.jpg', aligned_dot)

    print("Task 1 complete: Basic color image created")
    print("Task 2 complete: Aligned images created")
    print("Offsets using NCC:", offsets_ncc)
    print("Offsets using simple dot product:", offsets_dot)

    # Task 3: Pyramid alignment (for large images)
    # For now, just use the same image as an example
    # The pyramid_align function returns only two values: aligned_image, offsets

    # Use the original pyramid_align function (renamed to pyramid_align_original)
    aligned_pyramid, offsets = pyramid_align_original(
        red_channel, green_channel, blue_channel, levels=3, scale_factor=4, offset_range=10
    )

    # Save the pyramid-aligned image
    plt.imsave('aligned_image_pyramid.jpg', aligned_pyramid)

    print("Task 3 complete: Pyramid-aligned image created")
    # Print the offsets instead of level_offsets and total_offsets
    print("Offsets:", offsets)

    # ... (Rest of your code) ...


def pyramid_align_original(red_channel, green_channel, blue_channel, levels=3, scale_factor=4, offset_range=10, use_center=True):
    """
    Align RGB channels using an image pyramid approach.
    """
    # Create pyramids
    red_pyramid = create_pyramid(red_channel, levels, scale_factor)
    green_pyramid = create_pyramid(green_channel, levels, scale_factor)
    blue_pyramid = create_pyramid(blue_channel, levels, scale_factor)

    # Initial offsets at coarsest level
    g_offset = (0, 0)
    b_offset = (0, 0)

    for level in range(levels):
        # Get current level images
        r = red_pyramid[level]
        g = np.roll(green_pyramid[level], g_offset, axis=(0, 1))
        b = np.roll(blue_pyramid[level], b_offset, axis=(0, 1))

        # Find offset at this level
        g_dy, g_dx = best_offset_ncc(r, g, offset_range, use_center)
        b_dy, b_dx = best_offset_ncc(r, b, offset_range, use_center)

        # Update offset (accumulate and scale to next level)
        if level != levels - 1:
            g_offset = (g_offset[0] + g_dy * scale_factor, g_offset[1] + g_dx * scale_factor)
            b_offset = (b_offset[0] + b_dy * scale_factor, b_offset[1] + b_dx * scale_factor)
        else:
            # Final offset update (no upscaling needed)
            g_offset = (g_offset[0] + g_dy, g_offset[1] + g_dx)
            b_offset = (b_offset[0] + b_dy, b_offset[1] + b_dx)

    # Apply final offsets to original channels
    aligned_green = np.roll(green_channel, g_offset, axis=(0, 1))
    aligned_blue = np.roll(blue_channel, b_offset, axis=(0, 1))

    # Stack channels in RGB order
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    # Normalize if needed
    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    offsets = {
        'green': g_offset,
        'blue': b_offset
    }

    return aligned_image, offsets

if __name__ == "__main__":
    main()

Processing image: https://github.com/Mian56/MV_mod4/blob/main/prokudin-gorskii/prokudin-gorskii/00351v.jpg?raw=true
Blue channel shape: (341, 396, 3)
Green channel shape: (341, 396, 3)
Red channel shape: (341, 396, 3)
Task 1 complete: Basic color image created
Task 2 complete: Aligned images created
Offsets using NCC: {'green': (15, 15), 'blue': (15, 15)}
Offsets using simple dot product: {'green': (-10, 8), 'blue': (-2, -2)}
Task 3 complete: Pyramid-aligned image created
Offsets: {'green': (10, 62), 'blue': (10, 38)}
